In [1]:
import pandas as pd
import glob
import requests as r
import os
import json
import jinja2
import re

In [2]:
template_loader = jinja2.FileSystemLoader(searchpath="./../backend/templates")
template_env = jinja2.Environment(loader=template_loader)
template_file = "report.html"
template = template_env.get_template(template_file)

In [3]:
users = pd.read_csv('./user_organisation.csv')
emails = users['email'].str.lower().tolist()
files = glob.glob('./**/*.xlsx', recursive=True)
form_dumps = './old_forms.json'

In [4]:
def rename_columns(column_names):
    new_column_names = {}
    for col in list(filter(lambda x: '|' in x, list(column_names))):
        new_column_name = col.split('|')[0]
        new_column_names.update({col: new_column_name})
        if "--OTHER" in col:
            new_column_names.update({col: f"{new_column_name}--OTHER"})
    return new_column_names

In [5]:
def get_all_forms():
    if os.path.isfile(form_dumps):
        with open(form_dumps, 'r') as f:
            return json.load(f)
    dumps = []
    for f in files:
        form_id = f.split("-")[-1].split(".xlsx")[0].split(" ")[0]
        instances = ["idh","isco"]
        for instance in instances:
            generated_id = r.get(f"https://webform.akvo.org/api/generate/{instance}/{form_id}")
            generated_id = generated_id.text
            form = r.get(f"https://webform.akvo.org/api/form/{generated_id}")
            if form.status_code == 200:
                dumps.append({
                    "form_id": form_id,
                    "api": f"https://webform.akvo.org/api/form/{generated_id}",
                    "instance": instance,
                    "forms": form.json(),
                    "file": f
                })
    with open(form_dumps, 'w', encoding='utf-8') as f:
        json.dump(dumps, f, indent=4)
    return dumps

In [6]:
def generate_payload(payload, form_detail):
    results = []
    for d in payload:
        detail = []
        org = users[users["email"].str.contains(d["Submitter"].lower())]
        result = {
            "id": d["Instance"],
            "uuid": d["Identifier"],
            "submitted": d["Submission Date"],
            "submitted_by": {
                "id": None,
                "name": d["Submitter"].lower()
            },
            "organisation": {
                "id": None,
                "name": None
            },
            "form": form_detail["form"]
        }
        if org.shape[0] > 0:
            org = org.to_dict("records")[0]
            result.update({
                "organisation": {
                    "id": org["organisation_id"],
                    "name": org["organisation_name"]
                },
                "submitted_by": {
                    "id": org["user_id"],
                    "name": org["email"]
                },

            })
        for fd in form_detail["detail"]:
            answers = []
            if not fd["repeatable"]:
                repeat = []
                for a in fd["answers"]:
                    value = d.get(a["id"])
                    if value != value:
                        value = False
                    if a["value_type"] == "list" and value:
                        value = value.split("|")
                        if d.get(a["id"] + "--OTHER"):
                            other = d.get(a["id"] + "--OTHER")
                            if other == other:
                                value.append("Other:" + d.get(a["id"] + "--OTHER"))
                    if a["value_type"] != "list" and value and a["unit"]:
                        value = str(value) + " " + a["unit"]
                    repeat.append({
                        "question": a["question"],
                        "value": value,
                        "value_type": a["value_type"],
                        "tooltip": a["tooltip"],
                        "unit": a["unit"]
                    })
                answers.append({"answers": repeat})
            else:
                values = d.get("repeat_group")
                for i, val in enumerate(values):
                    repeat = []
                    for a in fd["answers"]:
                        value = val.get(a["id"])
                        if value:
                            if value != value:
                                value = False
                            if a["value_type"] == "list" and value:
                                value = value.split("|")
                                if val.get(a["id"] + "--OTHER"):
                                    other = val.get(a["id"] + "--OTHER")
                                    if other == other:
                                        value.append("Other:" + other)
                            if a["value_type"] != "list" and value and a["unit"]:
                                value = str(value) + " " + a["unit"]
                            repeat.append({
                                "question": a["question"],
                                "value": value,
                                "value_type": a["value_type"],
                                "tooltip": a["tooltip"],
                                "unit": a["unit"]
                            })
                    if len(repeat):
                        answers.append({"answers": repeat})
                answers = [{"answers": a["answers"], "repeat": i} for i, a in enumerate(answers)]
            detail.append({
                "name": fd["name"],
                "repeatable": fd["repeatable"],
                "data": answers
            })
        result.update({"detail":detail})
        results.append(result)
    return results

In [7]:
forms = get_all_forms()
transformed_forms = []
for form in forms:
    questionGroups = []
    for questionGroup in form['forms']["questionGroup"]:
        questions = []
        for question in questionGroup["question"]:
            tooltip = False
            unit = False
            if question.get("help"):
                tooltip = question.get("help").get("text")
                if "##" in tooltip:
                    unit = tooltip.split("##")[2]
                    tooltip = tooltip.split("##")[0]
            questions.append({
                "id":question["id"].split("Q")[1],
                "question": question["text"].replace("\n",""),
                "value_type": "list" if question["type"] == "option" else "text",
                "tooltip": tooltip,
                "unit": unit
            })
        questionGroups.append({
            "name": questionGroup["heading"],
            "repeatable":questionGroup["repeatable"],
            "answers": questions,
            "question_ids": [q["id"] for q in questions]
        })
    transformed_forms.append({
        "file": form["file"],
        "form": {
            "name": form['forms']["name"].replace("_"," "),
            "id": form['form_id']
        },
        "detail": questionGroups
    })

In [8]:
for file in files:
    excel = pd.ExcelFile(file)
    data = pd.read_excel(file, 'Raw Data', skiprows=1)
    repeat_group_answers = []
    repeat_group_sheets = list(filter(lambda x: 'Raw Data' != x, excel.sheet_names))
    for sheet in repeat_group_sheets:
        df = pd.read_excel(file, sheet, skiprows=1)
        df = df.rename(columns=rename_columns(list(df)))
        df = df.to_dict('records')
        for d in df:
            repeat_group_answers.append(d)
    data = data.rename(columns=rename_columns(list(data))).to_dict('records')
    for d in data:
        repeat_group = list(filter(lambda x: x['Identifier'] == d['Identifier'], repeat_group_answers))
        d.update({'repeat_group': repeat_group})
    selected_form = list(filter(lambda x: x["file"] == file, transformed_forms))[0]
    data = generate_payload(data, selected_form)
    for res in data:
        if res["organisation"]["name"]:
            output_text = template.render(
                webdomain="https://isco.akvo.org",
                data=res,
                detail=res["detail"])
            organisation_name = re.sub(
                '[^A-Za-z0-9_]+', '',
                res["organisation"]["name"].lower().replace(" ", "_"))
            organisation_id = res["organisation"]["id"]
            organisation_folder = f"{organisation_id}_{organisation_name}"
            if not os.path.exists(f"./results/{organisation_folder}"):
                os.makedirs(f"./results/{organisation_folder}")
            filename = "{}-{}".format(
                res["form"]["id"],
                res["id"]
            )
            html_path = "./results/{}/{}.html".format(
                organisation_folder,
                filename)
            html_file = open(html_path, 'w', encoding='utf-8')
            html_file.write(output_text)